# 03 - ré-Processamento e Construção do Pipeline (scikit-learn)

<a href="../README.md" title="Voltar para a página principal">
🏠 Voltar para Home
</a>

## Visão Geral

Este notebook implementa o *pré-processamento completo* do dataset para modelos de Machine Learning, seguindo boas práticas recomendadas pela biblioteca scikit-learn.

## O pipeline final incluirá:

1. Separação entre features numéricas e categóricas
2. Imputação de valores ausentes
3. Codificação de variáveis categóricas (One-Hot Encoding)
4. Padronização de variáveis numéricas
5. Montagem do ColumnTransformer
6. Geração do conjunto final X_prepared e y

Este notebook prepara os dados para os próximos passos:

- 04_modelos_classicos.ipynb
- 05_modelos_avancados.ipynb

# Importações

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder
)
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

# Trabalho com html
from IPython.display import display, HTML

# 1. Carregamento e Visualização Preliminar

***Descrição:*** Utilizando o dataset de acompanhamento operacional dos pedidos tratados.
> **Arquivo e:** database/processed/acompanhamento_operacional_FE.csv

In [12]:
file_path = "../database/processed/acompanhamento_operacional_FE.csv"

df = pd.read_csv(file_path)

print("Dataset carregado com sucesso!")
print("Formato:", df.shape)
df.head()

Dataset carregado com sucesso!
Formato: (488398, 29)


,sigla_cliente,tipo_veiculo,qtde_itens,volume,peso,m3,uf,fl_base,representante,flag_entrega_agendada,...,horas_conferencia,horas_emissao,horas_analise_producao,horas_minuta,horas_exped_minuta,hora_analise_transporte,lead_time_total_horas,complexidade_operacional,pedido_grande_flag,processo_longo_flag
0,NTL,TRUCK 70 M3,1,1,144.30,2.10,MS,0,NÃO DEFINIDO,Não,...,0.0,0.0,22.0,0.0,16.0,161.0,229.0,1,0,0
1,NTL,TRUCK 70 M3,1,1,130.30,2.33,MS,0,NÃO DEFINIDO,Não,...,0.0,0.0,22.0,0.0,16.0,165.0,234.0,1,0,0
2,NTL,TRUCK 70 M3,1,1,97.00,2.22,SC,0,SC CARGO TRANSPORTES LTDA,Não,...,0.0,0.0,0.0,0.0,16.0,40.0,249.0,1,0,0
3,MMM,TRUCK 75 M3,7,43,182.79,1.08,CE,0,VELOMAX BRASIL TRANSPORTES LTD,Não,...,1.0,0.0,11.0,99.0,8.0,93.0,266.0,7,1,0
4,MMM,TRUCK 70 M3,4,9,25.50,0.07,CE,0,MFM TRANSPORTES,Sim,...,1.0,0.0,12.0,172.0,16.0,285.0,542.0,4,1,1


# 2. Identificar Variáveis Numércias e Categóricas

In [13]:
TARGET = "fl_atraso_cli"

# Remover target para análise inicial
cols = df.columns.tolist()

num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
num_cols = [c for c in num_cols if c != TARGET]

cat_cols = df.select_dtypes(include=['object']).columns.tolist()

# Cria o DataFrame-resumo
df_summary = pd.DataFrame({
    'tipo': ['numerico', 'categorico'],
    'qtde': [len(num_cols), len(cat_cols)],
    'features': [num_cols, cat_cols]
})

html = f"""
<div style="display: flex; gap: 50px; justify-content: left;">

    <div>
        <h3 style="text-align:center;">Summary</h3>
        {df_summary.to_html(index=True)}
    </div>

</div>
"""

display(HTML(html))

,tipo,qtde,features
0,numerico,22,"[qtde_itens, volume, peso, m3, fl_base, qtde_ocams, peso_cubado_rodoviario, horas_pre_conferencia, horas_distribuicao_cotas, horas_planejamento, horas_divisao_ocam, horas_coleta, horas_conferencia, horas_emissao, horas_analise_producao, horas_minuta, horas_exped_minuta, hora_analise_transporte, lead_time_total_horas, complexidade_operacional, pedido_grande_flag, processo_longo_flag]"
1,categorico,6,"[sigla_cliente, tipo_veiculo, uf, representante, flag_entrega_agendada, modalidade]"


# 3. Estratégia de Imputação (numéricas e categóricas)

**Descrição:** a estratégia de imputação refere-se aos métodos usados para preencher valores ausentes (dados faltantes) em um conjunto de dados. A escolha do método depende do tipo de dados (numéricos ou categóricos), do volume de dados faltantes e do padrão de ausência, visando minimizar o viés e maximizar a precisão do modelo final.

**Estratégias recomendadas:**
- **Numéricas:** Usar median por ser robusto contra outliers e a mediana não é afetada por caudas pesadas.
- **Categóricas:** Usar most_frequent (moda), pois evita criar categoria artificial, mantém coerência sendo o padrão mais recomendado para categorias como UF, modalidade, sigla do cliente etc.

In [14]:
# Imputação numérica: mediana -> robusta a outliers
numeric_imputer = SimpleImputer(strategy="median")

# Imputação categórica: moda -> mantém consistência
categorical_imputer = SimpleImputer(strategy="most_frequent")

print("Imputadores definidos com sucesso!")


Imputadores definidos com sucesso!


# 4. Codificação Categórica (OneHot Encoding)

**Descrição:** A etapa de codificação categórica transforma variáveis textuais em representações numéricas adequadas para modelos de Machine Learning.  
Modelos matemáticos não conseguem interpretar textos como “SP”, “Rodoviário”, “HR” ou “Cliente X”. Por isso, utilizamos o OneHotEncoder, que converte cada categoria em uma coluna binária (0 ou 1).

**Usando OneHotEncoder:** 
É o método mais seguro e universal para modelos clássicos, como Regressão Logística, Árvores, Random Forest e Gradient Boosting.  
- **O parâmetro handle_unknown="ignore"** evita falhas durante o deploy caso surja uma categoria nova.  
- **O parâmetro sparse_output=False** permite visualizar o dataframe transformado em formato denso, facilitando debug, salvamento e inspeção.  

**Benefícios:**

- Elimina a escala arbitrária de categorias.
- Evita ordinalidade falsa (ex.: SP > RJ?).
- Permite ao modelo capturar relações específicas entre categorias e o atraso.
- Torna o pipeline mais robusto para uso em produção.

In [15]:
# 4. Encoding Categórico (OneHot)
categorical_encoder = OneHotEncoder(
    handle_unknown="ignore",
    sparse_output=False
)

print("OneHotEncoder configurado!")

OneHotEncoder configurado!


# 5. Scaling (Padronização das Variáveis Numéricas)

**Descrição:** A padronização é uma etapa fundamental na preparação dos dados, especialmente para modelos sensíveis à escala das variáveis. Neste projeto, utilizamos o StandardScaler, que transforma todas as variáveis numéricas para uma escala comum, com média 0 e desvio padrão 1.

**Motivações para o uso do StandardScaler:**
- Variáveis como volume, peso e horas operacionais possuem escalas muito diferentes.
- Modelos como Regressão Logística, SVM, KNN e Redes Neurais exigem escalas comparáveis para funcionar adequadamente.
- A padronização evita que o modelo interprete erroneamente variáveis com escala maior como sendo mais importantes.

**Vantagens:**
- Aumenta a estabilidade numérica do modelo.
- Melhora a convergência de algoritmos baseados em gradiente.
- Reduz o viés provocado por features com magnitude exagerada.

Assim como no imputer e no encoder, aqui configuramos o StandardScaler, mas sua aplicação real acontece somente dentro do Pipeline, durante o .fit(), garantindo transformação consistente tanto no treino quanto na inferência.

In [16]:
# 5. Scaling numérico
numeric_scaler = StandardScaler()

print("Scaler definido!")

Scaler definido!


# 6. ColumnTransformer — Unindo o Processo

**Descrição:** Após definir os componentes individuais (imputers, encoder e scaler), esta etapa consolida todos esses elementos dentro de um único bloco de pré-processamento usando o ColumnTransformer.

Cada grupo de variáveis (numéricas e categóricas) recebe seu próprio pipeline especializado:

**Pipeline Numérico:**
- Imputação por mediana
- Padronização com StandardScaler

**Ppeline Categórico:**
- Iputação por moda
- Codificação com OneHotEncoder

O ColumnTransformer garante que cada transformação seja aplicada apenas ao tipo correto de variável, preservando integridade, modularidade e reprodutibilidade do pipeline.  
A aplicação prática das transformações ocorrerá somente no momento do fit() do modelo, onde o pipeline executa imputação, codificação e escala em um só fluxo, produzindo dados consistentes tanto no treino quanto na inferência.

In [17]:
# 6. Construção do ColumnTransformer
preprocess = ColumnTransformer(transformers=[
    ("num_pipeline", Pipeline(steps=[
        ("imputer", numeric_imputer),
        ("scaler", numeric_scaler)
    ]), num_cols),

    ("cat_pipeline", Pipeline(steps=[
        ("imputer", categorical_imputer),
        ("encoder", categorical_encoder)
    ]), cat_cols)
])

print("ColumnTransformer criado com sucesso!")

ColumnTransformer criado com sucesso!


# 7. Split Train/Test

**Descrição:** Nesta etapa, dividimos o dataset em dois subconjuntos principais:

**Treino (70%):** Usado para ajustar o pipeline de pré-processamento e treinar os modelos.

**Teste (20%):** Usado apenas após a modelagem, para medir o desempenho real do modelo em dados nunca vistos.

Utilizamos stratify=y para manter a proporção original de atrasos/não atrasos, garantindo que o conjunto de treino e teste representem adequadamente o problema.

Essa separação impede vazamento de informação e garante avaliação honesta da performance dos modelos.

In [18]:
# 7. Train/Test Split
X = df.drop(columns=[TARGET])
y = df[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

print("Split realizado!")
print("Treino:", X_train.shape)
print("Teste:", X_test.shape)

Split realizado!
Treino: (341878, 28)
Teste: (146520, 28)


# 8. Pipeline Final (Pré-processamento completo)

**Descrição:** Nesta etapa construímos o pipeline completo de modelagem, combinando:

- imputação de valores ausentes
- codificação categórica (OneHotEncoder)
- padronização de variáveis numéricas
- integração via ColumnTransformer

O uso de um Pipeline garante reprodutibilidade, prevenção de vazamento de informação e padronização do fluxo de transformação.
Agora, quando executarmos .fit(), todo o pré-processamento será aplicado automaticamente antes do treinamento do modelo.

In [19]:
# 8. Pipeline Final de Pré-processamento
pipeline_preprocess = Pipeline(steps=[
    ("preprocess", preprocess)
])

print("Pipeline de pré-processamento pronto!")

Pipeline de pré-processamento pronto!
